## Step 1: Invoke Bedrock Models to get _Inferences_ on a user provided Dataset
---

This notebook does as follows:

1. Generates inferences on a user provided dataset, using Foundation models on Amazon Bedrock

1. Uses [Litellm](https://www.litellm.ai/) as an interface to interact with the Bedrock API

1. Uses `Ray`, which is used to run inferences in an asynchronous manner

1. Records metrics like the `p90, p95` latency, `prompt token counts`, `completion token counts`, and more.

1. Saves all the combined _model responses_ to user questions from the source dataset in a `all_results.csv` that is used later in the _evaluation step_ for the evaluation process.

In [1]:
# import the libraries
import os
import ray
import json
import yaml
import time
import boto3
import logging
import textwrap
import pandas as pd
from pathlib import Path
from functools import reduce
from litellm import completion
from typing import Dict, List, Optional

In [2]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# initialize the ray service to run async calls in parallel to bedrock easily
if ray.is_initialized():
    ray.shutdown()
ray.init()

2024-06-07 13:30:43,980	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.11.7
Ray version:,2.10.0


In [4]:
# global constants
CONFIG_FILE_PATH = "config.yaml"

# read the config yaml file
fpath = CONFIG_FILE_PATH
with open(fpath, 'r') as yaml_in:
    config = yaml.safe_load(yaml_in)
logger.info(f"config read from {fpath} -> {json.dumps(config, indent=2)}")

[2024-06-07 13:30:45,133] p95821 {2927026569.py:8} INFO - config read from config.yaml -> {
  "app_name": "llm-as-a-judge-eval-pipeline",
  "aws": {
    "region": "us-east-1"
  },
  "run_steps": {
    "1_get_inference.ipynb": true,
    "2_get_llm_as_a_judge_eval.ipynb": true
  },
  "pdf_dir_info": {
    "data_dir": "data",
    "dataset_dir": "source_data",
    "dataset_file_name": "data_user_system_prompt_version.csv",
    "metrics": "results",
    "llm_as_a_judge_dir": "eval_completions",
    "prompt_dir": "prompt_template",
    "llm_as_a_judge_completions": "llm_as_a_judge_completions.csv",
    "raw_llm_as_a_judge_completions": "raw_llm_responses.csv",
    "llm_as_a_judge_comparisons": "llm_as_a_judge_comparisons.csv",
    "llm_comparisons_txt": "llm_as_a_judge_comparisons.txt",
    "llm_as_a_judge_pick_rate": "llm_as_a_judge_pick_rate.csv",
    "eval_prompt_template": "llama3_eval_prompt.txt",
    "prompt_template": "prompt_template.txt",
    "processed_eval_prompts": "processed_eva

In [5]:
# initialize all global variables that are used across this notebook hydrated from the `config.yaml` file

# name of your source xlsx/xls/csv file 
FILE_NAME: str = config['pdf_dir_info']['dataset_file_name']
# data directory
DATA_DIR: str = config['pdf_dir_info']['data_dir']
FILE_RELATIVE_PATH: str = os.path.join(config['pdf_dir_info']['dataset_dir'], FILE_NAME)
INPUT_FPATH: str = os.path.join(DATA_DIR, FILE_RELATIVE_PATH)
USER_PROMPT_COL: str = config['dataset_info']['user_question_col']
SYSTEM_PROMPT_COL: str = config['dataset_info']['system_prompt_col']
INFERENCE_PARAMETERS: Dict = config['inference_parameters']
ON_LIST = list(filter(None, [USER_PROMPT_COL, 
                            SYSTEM_PROMPT_COL]))

# result files
ALL_RESULTS_FPATH = os.path.join(DATA_DIR, config['pdf_dir_info']['metrics'])
INFERENCE_LATENCY_SUMMARY_FPATH = os.path.join(ALL_RESULTS_FPATH, config['pdf_dir_info']['inference_latency_summary_fname'])
bedrock_model_ids: List[str] = config['bedrock_fms_to_test']

### Get Inference for the given dataset
---

This portion of the notebook gets inference using `Ray` (which is used to handle asynchronous calls to `Litellm`) to get inferences from the user questions in the given dataset

In [6]:
def generate_task_inference(model_id: str, 
                            user_prompt: str, 
                            system_prompt: str, 
                            inference_parameters: Dict) -> Dict:
    """
    This function takes in a dictionary (which contains information on the user data and prompts) 
    to generate inference using a bedrock model id, and returns a dictionary containing the model 
    completion, and latency (in seconds).
    """
    # represents the service name
    service_name: str = "bedrock"
    temperature = inference_parameters.get('temperature', 0.1)
    caching = inference_parameters.get('caching', False)
    max_tokens = inference_parameters.get("max_tokens", 500)
    # represents creating the bedrock model to invoke the litellm api for response for titan, llama and claude
    bedrock_model: str = f"{service_name}/{model_id}"
    # represents the current aws region
    aws_region = boto3.Session().region_name
    # initialize the response dict
    ret = dict(user_prompt=user_prompt,
               system_prompt=system_prompt,
               completion=None,
               model_id=model_id,
               time_taken_in_seconds=None,
               prompt_token_count=None,
               completion_token_count=None,
               exception=None)
    # custom messages formatting for when the user/system roles are given together
    if config['dataset_info']['system_prompt_col'] is not None:
        messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
        ]
        print(f"messages: {messages}")
    else:
        body = ret['user_prompt']
        messages=[{ "content": body, "role": "user"}]

    # set the env var for aws_region
    os.environ["AWS_REGION_NAME"] = aws_region 
    try:
        print(f"Invoking {bedrock_model}......")
        response = completion(model=bedrock_model,
                              messages=messages,
                              temperature=temperature,
                              max_tokens=max_tokens,
                              caching=caching)

        # iterate through the entire model response
        for idx, choice in enumerate(response.choices):
            print(f"choice {idx+1} of {len(response.choices)} ")
            # extract the message and the message's content from litellm
            if choice.message and choice.message.content:
                # extract the response from the dict
                ret["completion"] = choice.message.content.strip()

        # Commenting out the code below that records the number of input and output tokens.
        # Extract number of input and completion prompt tokens (this is the same structure for embeddings and text generation models on Amazon Bedrock)
        ret['prompt_token_count'] = response.usage.prompt_tokens
        ret['completion_token_count'] = response.usage.completion_tokens
        # Extract latency in seconds
        latency_ms = response._response_ms
        ret['time_taken_in_seconds']  = latency_ms / 1000
    except Exception as e:
        print(f"Exception occurred during invoking {model_id}, exception={e}")
        ret['exception'] = e
    return ret

In [7]:
@ray.remote
def async_generate_task_inference(input: Dict, model_id: str, inference_parameters: Dict) -> Dict:
    resp = generate_task_inference(model_id, input.get('user_prompt'), input.get('system_prompt'), inference_parameters)
    resp_this_model = {"model_id": model_id,
                       USER_PROMPT_COL: input.get('user_prompt'),
                       SYSTEM_PROMPT_COL: input.get('system_prompt'),
                       f"{model_id}-response": resp['completion'],
                       f"{model_id}-time_taken_in_seconds": resp['time_taken_in_seconds'],
                       f"{model_id}-prompt_token_count": resp['prompt_token_count'],
                       f"{model_id}-completion_token_count": resp['completion_token_count'],
                       f"{model_id}-exception": resp['exception']}
    return resp_this_model

In [8]:
logger.info(f"File name to be processed: {INPUT_FPATH}")
data_file = Path(INPUT_FPATH)
if data_file.suffix == '.csv':
    logger.info(f"processing the csv file: {data_file}")
    original_eval_df = pd.read_csv(data_file)
elif data_file.suffix in ['.xls', '.xlsx']:
    logger.info(f"processing the xls/xlsx file: {data_file}")
    original_eval_df = pd.read_excel(data_file)
else:
    raise ValueError(f"Unsupported file format: {data_file.suffix}")
logger.info(f"input data frame shape is {original_eval_df.shape}")
# drop the columns that have all 'NaN' values
original_eval_df = original_eval_df.dropna(axis=1, how='all')
original_eval_df.head(10)

[2024-06-07 13:30:45,575] p95821 {3661773823.py:1} INFO - File name to be processed: data/source_data/data_user_system_prompt_version.csv


[2024-06-07 13:30:45,576] p95821 {3661773823.py:4} INFO - processing the csv file: data/source_data/data_user_system_prompt_version.csv


[2024-06-07 13:30:45,584] p95821 {3661773823.py:11} INFO - input data frame shape is (10, 3)


,user_input,system_prompt,model_1
0,What is the Heisenberg uncertainty principle?,The Heisenberg uncertainty principle is a fund...,The Heisenberg uncertainty principle states th...
1,What is the Schrödinger equation and how is it...,The Schrödinger equation is a fundamental equa...,The Schrödinger equation is a fundamental equa...
2,What is the greenhouse effect and how does it ...,The greenhouse effect is a natural process tha...,The greenhouse effect is a natural process tha...
3,What is the photoelectric effect and how did i...,The photoelectric effect is a phenomenon in wh...,"When light shines on a metal, electrons can be..."
4,What is the structure of the atom and how was ...,The structure of the atom has been a fundament...,"Modern atomic models, based on quantum mechani..."
5,What is the role of catalysts in chemical reac...,Catalysts are substances that increase the rat...,A catalyst is a substance that can be added to...
6,What is the second law of thermodynamics and h...,The second law of thermodynamics is one of the...,The second law of thermodynamics states that t...
7,What is the difference between nuclear fission...,Nuclear fission and nuclear fusion are two fun...,The phenomenon of nuclear fission. Fission occ...
8,What is the difference between classical mecha...,Classical mechanics and quantum mechanics are ...,Classical mechanics describes the physics of m...
9,What is the difference between endothermic and...,"In chemistry, chemical reactions can be classi...",If you touch a container that holds an endothe...


In [9]:
original_eval_list = json.loads(original_eval_df.to_json(orient='records'))
original_eval_list

[{'user_input': 'What is the Heisenberg uncertainty principle?',
  'system_prompt': 'The Heisenberg uncertainty principle is a fundamental principle in quantum mechanics that states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously. This principle arises from the wave-particle duality of quantum particles and has profound implications for our understanding of the behavior of matter at the atomic and subatomic scales',
  'model_1': 'The Heisenberg uncertainty principle states that the position and momentum of a particle cannot be measured precisely at the same time, reflecting the wave-particle duality of quantum particles.'},
 {'user_input': 'What is the Schrödinger equation and how is it used in quantum mechanics?',
  'system_prompt': "The Schrödinger equation is a fundamental equation in quantum mechanics that describes the behavior of particles at the quantum le

In [10]:
# list of the bedrock model ids that are used in generating inferences
bedrock_model_ids: List[str] =[d['model_id'] for d in config['bedrock_fms_to_test']]
bedrock_model_ids

['anthropic.claude-3-haiku-20240307-v1:0',
 'anthropic.claude-3-sonnet-20240229-v1:0']

### Run the inferences to get model responses in parallel using `Ray`

In [11]:
erroneous_count: int = 0
resp_list = []
n: int = config.get('parallel_inference_count')
st_overall = time.perf_counter()
# Iterate over each bedrock model ID
for model_id in bedrock_model_ids:
    logger.info(f"going to get inference from model={model_id}")
    list_of_lists = [original_eval_list[i * n:(i + 1) * n] for i in range((len(original_eval_list) + n - 1) // n )]
    st = time.perf_counter()
    for idx, sublist in enumerate(list_of_lists):
        logger.info(f"processing sublist={idx+1}/{len(list_of_lists)} for model_id={model_id}")
        for input in sublist:
            print(f"input logged: {input}")
            try:
                input_dict = dict(user_prompt=input.get(USER_PROMPT_COL), system_prompt=input.get(SYSTEM_PROMPT_COL))
                result = ray.get(async_generate_task_inference.remote(input_dict, model_id, INFERENCE_PARAMETERS))
                resp_list.append(result)
            except Exception as e:
                logger.error(f"Error processing input: {input} for model_id={model_id}, error: {e}")
                erroneous_count += 1

    elapsed_time = time.perf_counter() - st
    logger.info(f"total time taken for {len(original_eval_list)} with model={model_id} is {elapsed_time:0.2f}")
elapsed_time = time.perf_counter() - st_overall
logger.info(f"total time taken for {len(original_eval_list)} with models={bedrock_model_ids} is {elapsed_time:0.2f}")
logger.info(f"total erroneous count: {erroneous_count}")

[2024-06-07 13:30:45,640] p95821 {1293849496.py:7} INFO - going to get inference from model=anthropic.claude-3-haiku-20240307-v1:0


[2024-06-07 13:30:45,641] p95821 {1293849496.py:11} INFO - processing sublist=1/2 for model_id=anthropic.claude-3-haiku-20240307-v1:0


input logged: {'user_input': 'What is the Heisenberg uncertainty principle?', 'system_prompt': 'The Heisenberg uncertainty principle is a fundamental principle in quantum mechanics that states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously. This principle arises from the wave-particle duality of quantum particles and has profound implications for our understanding of the behavior of matter at the atomic and subatomic scales', 'model_1': 'The Heisenberg uncertainty principle states that the position and momentum of a particle cannot be measured precisely at the same time, reflecting the wave-particle duality of quantum particles.'}


input logged: {'user_input': 'What is the Schrödinger equation and how is it used in quantum mechanics?', 'system_prompt': "The Schrödinger equation is a fundamental equation in quantum mechanics that describes the behavior of particles at the quantum level. It was formulated by the Austrian physicist Erwin Schrödinger in 1925 and is considered one of the most important equations in physics.\n\nThe Schrödinger equation is a partial differential equation that relates the wave function, which contains all the information about the quantum state of a particle, to the particle's energy and potential energy. The wave function is a mathematical representation of the particle's behavior and describes the probability of finding the particle in a particular state or position.\n\nThe Schrödinger equation is written as:\n\niℏ(∂Ψ/∂t) = ĤΨ\n\nWhere:\n- i is the imaginary unit (√-1)\n- ℏ is the reduced Planck constant\n- Ψ (psi) is the wave function\n- Ĥ is the Hamiltonian operator, which represents

input logged: {'user_input': 'What is the greenhouse effect and how does it contribute to global warming?', 'system_prompt': "The greenhouse effect is a natural process that occurs when certain gases in the Earth's atmosphere, known as greenhouse gases, trap heat from the Sun's radiation and prevent it from escaping back into space. This process is essential for maintaining a habitable temperature on Earth, as without it, the planet's average surface temperature would be around -18°C (-0.4°F), which is too cold for most life forms to survive.\n\nThe primary greenhouse gases are water vapor (H2O), carbon dioxide (CO2), methane (CH4), nitrous oxide (N2O), and certain synthetic gases like chlorofluorocarbons (CFCs) and hydrofluorocarbons (HFCs). These gases absorb and re-emit infrared radiation, which is a form of thermal energy emitted by the Earth's surface after being heated by the Sun's visible and ultraviolet radiation.\n\nThe greenhouse effect works as follows:\n\n1. The Sun's radia

...

input logged: {'user_input': 'What is the difference between classical mechanics and quantum mechanics?', 'system_prompt': "Classical mechanics and quantum mechanics are two fundamental theories in physics that describe the behavior of particles and systems at different scales and under different conditions.\n\nClassical Mechanics:\nClassical mechanics is a branch of physics that deals with the motion and behavior of macroscopic objects, such as planets, stars, and everyday objects. It is based on the principles of Newtonian mechanics and is governed by Newton's laws of motion, as well as concepts like energy, momentum, and force.\n\nClassical mechanics assumes that particles and objects have well-defined positions and velocities at any given time, and their behavior can be predicted with certainty using deterministic equations of motion. It is a highly successful theory that accurately describes the motion of large-scale objects in everyday situations and has been instrumental in many

input logged: {'user_input': 'What is the difference between endothermic and exothermic reactions?', 'system_prompt': 'In chemistry, chemical reactions can be classified as either endothermic or exothermic based on the energy changes that occur during the reaction process. The main difference between endothermic and exothermic reactions lies in the direction of energy flow and the temperature changes associated with the reaction.\n\nEndothermic Reactions:\nEndothermic reactions are chemical processes that absorb energy from the surroundings in the form of heat. In an endothermic reaction, the reactants require an input of energy to break existing bonds and form new bonds, resulting in an increase in the overall energy of the system.\n\nEndothermic reactions are characterized by the following:\n\n1. Energy is absorbed from the surroundings, usually in the form of heat.\n2. The temperature of the surroundings decreases as energy is absorbed by the reaction.\n3. The products have a higher

[2024-06-07 13:32:56,292] p95821 {1293849496.py:23} INFO - total time taken for 10 with model=anthropic.claude-3-sonnet-20240229-v1:0 is 83.03


[2024-06-07 13:32:56,294] p95821 {1293849496.py:25} INFO - total time taken for 10 with models=['anthropic.claude-3-haiku-20240307-v1:0', 'anthropic.claude-3-sonnet-20240229-v1:0'] is 130.65


[2024-06-07 13:32:56,294] p95821 {1293849496.py:26} INFO - total erroneous count: 0


In [12]:
# view some responses generated
resp_list[:5]

[{'model_id': 'anthropic.claude-3-haiku-20240307-v1:0',
  'user_input': 'What is the Heisenberg uncertainty principle?',
  'system_prompt': 'The Heisenberg uncertainty principle is a fundamental principle in quantum mechanics that states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously. This principle arises from the wave-particle duality of quantum particles and has profound implications for our understanding of the behavior of matter at the atomic and subatomic scales',
  'anthropic.claude-3-haiku-20240307-v1:0-response': "The Heisenberg uncertainty principle states that there is a fundamental limit to the precision with which certain pairs of physical properties of a particle, such as position and momentum, can be known simultaneously.\n\nThe key points about the Heisenberg uncertainty principle are:\n\n1. It applies to pairs of complementary variables, such as

In [13]:
df_list = []
for model_id in bedrock_model_ids:
    df_list.append(pd.DataFrame([r for r in resp_list if r['model_id'] == model_id]).drop(['model_id'], axis=1))    
from functools import reduce
logger.info(f"on_list: {ON_LIST}")
try:
    # if the system prompt is separately provided, merge on that column too else, just use the user
    # column for the merge
    df_resp = reduce(lambda x, y: pd.merge(x, y, on=ON_LIST), 
                    df_list)
except Exception as e:
    logger.error(f"df was not merged: {e}")
logger.info(f"shape of response data frame={df_resp.shape}")

[2024-06-07 13:32:56,342] p95821 {1844015812.py:5} INFO - on_list: ['user_input', 'system_prompt']


[2024-06-07 13:32:56,346] p95821 {1844015812.py:13} INFO - shape of response data frame=(10, 12)


In [14]:
# view the data frame
df_resp = df_resp.loc[:, ~df_resp.columns.str.startswith('None')]
df_resp.head(10)

,user_input,system_prompt,anthropic.claude-3-haiku-20240307-v1:0-response,anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds,anthropic.claude-3-haiku-20240307-v1:0-prompt_token_count,anthropic.claude-3-haiku-20240307-v1:0-completion_token_count,anthropic.claude-3-haiku-20240307-v1:0-exception,anthropic.claude-3-sonnet-20240229-v1:0-response,anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds,anthropic.claude-3-sonnet-20240229-v1:0-prompt_token_count,anthropic.claude-3-sonnet-20240229-v1:0-completion_token_count,anthropic.claude-3-sonnet-20240229-v1:0-exception
0,What is the Heisenberg uncertainty principle?,The Heisenberg uncertainty principle is a fund...,The Heisenberg uncertainty principle states th...,3.131573,98,248,None,"The Heisenberg uncertainty principle, formulat...",5.701876,98,303,None
1,What is the Schrödinger equation and how is it...,The Schrödinger equation is a fundamental equa...,The Schrödinger equation is a fundamental equa...,4.926864,589,568,None,The Schrödinger equation is a fundamental equa...,7.400591,589,591,None
2,What is the greenhouse effect and how does it ...,The greenhouse effect is a natural process tha...,The greenhouse effect is a natural process tha...,5.654147,532,409,None,The greenhouse effect is a natural process tha...,8.838915,532,513,None
3,What is the photoelectric effect and how did i...,The photoelectric effect is a phenomenon in wh...,The photoelectric effect is a phenomenon in wh...,5.587937,516,518,None,The photoelectric effect is a phenomenon in wh...,14.179667,516,572,None
4,What is the structure of the atom and how was ...,The structure of the atom has been a fundament...,The structure of the atom has been determined ...,5.674264,510,493,None,The current model of the atomic structure is b...,9.992657,510,499,None
5,What is the role of catalysts in chemical reac...,Catalysts are substances that increase the rat...,Catalysts play a crucial role in chemical reac...,5.593801,472,387,None,The primary role of catalysts in chemical reac...,8.274840,472,375,None
6,What is the second law of thermodynamics and h...,The second law of thermodynamics is one of the...,The second law of thermodynamics states that t...,4.400713,527,336,None,The second law of thermodynamics is a fundamen...,8.548479,527,462,None
7,What is the difference between nuclear fission...,Nuclear fission and nuclear fusion are two fun...,The main differences between nuclear fission a...,3.910536,400,332,None,The main difference between nuclear fission an...,7.401660,400,350,None
8,What is the difference between classical mecha...,Classical mechanics and quantum mechanics are ...,The main differences between classical mechani...,3.582687,630,360,None,The main differences between classical mechani...,8.474581,630,373,None
9,What is the difference between endothermic and...,"In chemistry, chemical reactions can be classi...",The main difference between endothermic and ex...,4.250860,607,362,None,The main difference between endothermic and ex...,4.078007,607,220,None


In [15]:
# get the original/target responses if any and merge it with the current df
try: 
    if df_resp is not None and config['dataset_info']['pre_existing_response_col'] is not None:
        df_resp_all = pd.merge(left=df_resp, right=original_eval_df, how="left",
                            left_on=ON_LIST, 
                            right_on=ON_LIST)
except Exception as e:
    logger.error(f"Could not perform the merge with the original data frame: {e}")


In [16]:
# view the current data in the df
df_resp_all.head(10)

,user_input,system_prompt,anthropic.claude-3-haiku-20240307-v1:0-response,anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds,anthropic.claude-3-haiku-20240307-v1:0-prompt_token_count,anthropic.claude-3-haiku-20240307-v1:0-completion_token_count,anthropic.claude-3-haiku-20240307-v1:0-exception,anthropic.claude-3-sonnet-20240229-v1:0-response,anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds,anthropic.claude-3-sonnet-20240229-v1:0-prompt_token_count,anthropic.claude-3-sonnet-20240229-v1:0-completion_token_count,anthropic.claude-3-sonnet-20240229-v1:0-exception,model_1
0,What is the Heisenberg uncertainty principle?,The Heisenberg uncertainty principle is a fund...,The Heisenberg uncertainty principle states th...,3.131573,98,248,None,"The Heisenberg uncertainty principle, formulat...",5.701876,98,303,None,The Heisenberg uncertainty principle states th...
1,What is the Schrödinger equation and how is it...,The Schrödinger equation is a fundamental equa...,The Schrödinger equation is a fundamental equa...,4.926864,589,568,None,The Schrödinger equation is a fundamental equa...,7.400591,589,591,None,The Schrödinger equation is a fundamental equa...
2,What is the greenhouse effect and how does it ...,The greenhouse effect is a natural process tha...,The greenhouse effect is a natural process tha...,5.654147,532,409,None,The greenhouse effect is a natural process tha...,8.838915,532,513,None,The greenhouse effect is a natural process tha...
3,What is the photoelectric effect and how did i...,The photoelectric effect is a phenomenon in wh...,The photoelectric effect is a phenomenon in wh...,5.587937,516,518,None,The photoelectric effect is a phenomenon in wh...,14.179667,516,572,None,"When light shines on a metal, electrons can be..."
4,What is the structure of the atom and how was ...,The structure of the atom has been a fundament...,The structure of the atom has been determined ...,5.674264,510,493,None,The current model of the atomic structure is b...,9.992657,510,499,None,"Modern atomic models, based on quantum mechani..."
5,What is the role of catalysts in chemical reac...,Catalysts are substances that increase the rat...,Catalysts play a crucial role in chemical reac...,5.593801,472,387,None,The primary role of catalysts in chemical reac...,8.274840,472,375,None,A catalyst is a substance that can be added to...
6,What is the second law of thermodynamics and h...,The second law of thermodynamics is one of the...,The second law of thermodynamics states that t...,4.400713,527,336,None,The second law of thermodynamics is a fundamen...,8.548479,527,462,None,The second law of thermodynamics states that t...
7,What is the difference between nuclear fission...,Nuclear fission and nuclear fusion are two fun...,The main differences between nuclear fission a...,3.910536,400,332,None,The main difference between nuclear fission an...,7.401660,400,350,None,The phenomenon of nuclear fission. Fission occ...
8,What is the difference between classical mecha...,Classical mechanics and quantum mechanics are ...,The main differences between classical mechani...,3.582687,630,360,None,The main differences between classical mechani...,8.474581,630,373,None,Classical mechanics describes the physics of m...
9,What is the difference between endothermic and...,"In chemistry, chemical reactions can be classi...",The main difference between endothermic and ex...,4.250860,607,362,None,The main difference between endothermic and ex...,4.078007,607,220,None,If you touch a container that holds an endothe...


### Record the `p50` and `p95` inference latencies in a `txt` file

In [17]:
time_taken_in_seconds_cols = [c for c in df_resp_all.columns if 'time_taken_in_seconds' in c]
Latency_cols = [c for c in df_resp_all.columns if 'Latency ' in c]
all_latency_cols_of_interest = time_taken_in_seconds_cols + Latency_cols
summary = ""
for c in all_latency_cols_of_interest:
    quantiles = list(round(df_resp_all[c].quantile([0.5, 0.95]), 2))
    s = f"[p50, p95] for {c}={quantiles}\n"
    summary += s
    logger.info(s)
Path(INFERENCE_LATENCY_SUMMARY_FPATH).write_text(summary)

[2024-06-07 13:32:56,424] p95821 {4188724700.py:9} INFO - [p50, p95] for anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds=[4.66, 5.67]



[2024-06-07 13:32:56,425] p95821 {4188724700.py:9} INFO - [p50, p95] for anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds=[8.37, 12.3]



179

### Upload the overall results to a `results.csv` file

In [18]:
def wrap_text(df, width):
    """
    This function wraps the text in a specific cell to a given
    width
    """
    for col in df.columns:
        df[col] = df[col].apply(lambda x: '\n'.join(textwrap.wrap(str(x), width)))
    return df

In [19]:
df_resp_all = wrap_text(df_resp_all, width=40)
cols = list(df_resp_all.columns)
user_input_index = df_resp_all.columns.get_loc(config['dataset_info']['user_question_col'])
response_cols = [col for col in df_resp_all.columns if col.endswith('-response')]
for col in response_cols:
    cols.pop(cols.index(col))
# Reinsert the response columns right after the user_input column
for col in reversed(response_cols):
    cols.insert(user_input_index + 1, col)
df_resp_all = df_resp_all[cols]

# Save the DataFrame to a CSV file
os.makedirs(ALL_RESULTS_FPATH, exist_ok=True)
all_results_csv_fpath: str = os.path.join(ALL_RESULTS_FPATH, 
                                          config['pdf_dir_info']['all_results_file_name'])
df_resp_all.to_csv(all_results_csv_fpath, index=False)
df_resp_all.head(10)

,user_input,anthropic.claude-3-haiku-20240307-v1:0-response,anthropic.claude-3-sonnet-20240229-v1:0-response,system_prompt,anthropic.claude-3-haiku-20240307-v1:0-time_taken_in_seconds,anthropic.claude-3-haiku-20240307-v1:0-prompt_token_count,anthropic.claude-3-haiku-20240307-v1:0-completion_token_count,anthropic.claude-3-haiku-20240307-v1:0-exception,anthropic.claude-3-sonnet-20240229-v1:0-time_taken_in_seconds,anthropic.claude-3-sonnet-20240229-v1:0-prompt_token_count,anthropic.claude-3-sonnet-20240229-v1:0-completion_token_count,anthropic.claude-3-sonnet-20240229-v1:0-exception,model_1
0,What is the Heisenberg uncertainty\nprinciple?,The Heisenberg uncertainty principle\nstates t...,"The Heisenberg uncertainty principle,\nformula...",The Heisenberg uncertainty principle is\na fun...,3.131573,98,248,None,5.701876,98,303,None,The Heisenberg uncertainty principle\nstates t...
1,What is the Schrödinger equation and how\nis i...,The Schrödinger equation is a\nfundamental equ...,The Schrödinger equation is a\nfundamental equ...,The Schrödinger equation is a\nfundamental equ...,4.926864,589,568,None,7.400591,589,591,None,The Schrödinger equation is a\nfundamental equ...
2,What is the greenhouse effect and how\ndoes it...,The greenhouse effect is a natural\nprocess th...,The greenhouse effect is a natural\nprocess th...,The greenhouse effect is a natural\nprocess th...,5.654147,532,409,None,8.838915,532,513,None,The greenhouse effect is a natural\nprocess th...
3,What is the photoelectric effect and how\ndid ...,The photoelectric effect is a phenomenon\nin w...,The photoelectric effect is a phenomenon\nin w...,The photoelectric effect is a phenomenon\nin w...,5.587937,516,518,None,14.179667,516,572,None,"When light shines on a metal, electrons\ncan b..."
4,What is the structure of the atom and\nhow was...,The structure of the atom has been\ndetermined...,The current model of the atomic\nstructure is ...,The structure of the atom has been a\nfundamen...,5.674264,510,493,None,9.992657,510,499,None,"Modern atomic models, based on quantum\nmechan..."
5,What is the role of catalysts in\nchemical rea...,Catalysts play a crucial role in\nchemical rea...,The primary role of catalysts in\nchemical rea...,Catalysts are substances that increase\nthe ra...,5.593801,472,387,None,8.27484,472,375,None,A catalyst is a substance that can be\nadded t...
6,What is the second law of thermodynamics\nand ...,The second law of thermodynamics states\nthat ...,The second law of thermodynamics is a\nfundame...,The second law of thermodynamics is one\nof th...,4.400713,527,336,None,8.548479,527,462,None,The second law of thermodynamics states\nthat ...
7,What is the difference between nuclear\nfissio...,The main differences between nuclear\nfission ...,The main difference between nuclear\nfission a...,Nuclear fission and nuclear fusion are\ntwo fu...,3.910536,400,332,None,7.40166,400,350,None,The phenomenon of nuclear fission.\nFission oc...
8,What is the difference between classical\nmech...,The main differences between classical\nmechan...,The main differences between classical\nmechan...,Classical mechanics and quantum\nmechanics are...,3.582687,630,360,None,8.474581,630,373,None,Classical mechanics describes the\nphysics of ...
9,What is the difference between\nendothermic an...,The main difference between endothermic\nand e...,The main difference between endothermic\nand e...,"In chemistry, chemical reactions can be\nclass...",4.25086,607,362,None,4.078007,607,220,None,If you touch a container that holds an\nendoth...
